In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
sys.path.append(os.path.realpath('..'))

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import plotly.express as ex
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from copy import deepcopy
from scipy.integrate import solve_ivp

import cache_magic

from IPython.display import display
import ipywidgets.widgets as widgets

from tracking_v2.target import ConstantVelocityTarget, SingerTarget
from tracking_v2.kalman import LinearKalmanFilter
from tracking_v2.motion import ConstantVelocityModel, ConstantAccelerationModel, SingerAccelerationModel
from tracking_v2.sensor import GeometricSensor
from tracking_v2.evaluation import Runner, run_many, evaluate_many, plot_nees, evaluate_nees, \
                                   plot_error, evaluate_runner, plot_2d, plot_3d

from tracking.util import to_df

In [ ]:
class Runner:
    def __init__(self):
        self.reset()

    def reset(self, q_model=1, q_real=1):
        self.rng = np.random.default_rng()
        self.T = 3.0

        self.x = np.zeros((3, 1))
        self.v = np.array([float(30), 0, 0]).reshape((3, 1))
        self.q_model = q_model
        self.q_real  = q_real
        self.R = 1.0

        self.H = np.array([[1, 0, 0, 0, 0, 0],
                           [0, 1, 0, 0, 0, 0],
                           [0, 0, 1, 0, 0, 0]])

        T  = self.T
        
        self.F = np.array([[1, 0, 0, T, 0, 0],
                           [0, 1, 0, 0, T, 0],
                           [0, 0, 1, 0, 0, T],
                           [0, 0, 0, 1, 0, 0],
                           [0, 0, 0, 0, 1, 0],
                           [0, 0, 0, 0, 0, 1]])

        T3 = self.T**3 / 3
        T2 = self.T**2 / 2
        T  = self.T

        self.Q = np.array([[T3, 0,  0,  T2, 0  ,0 ],
                           [0,  T3, 0,  0,  T2, 0 ],
                           [0,  0,  T3, 0,  0,  T2],
                           [T2, 0,  0,  T,  0,  0 ],
                           [0,  T2, 0,  0,  T,  0 ],
                           [0,  0,  T2, 0,  0,  T ]]) * self.q_model**2


        # initialize
        self.x_hat = np.zeros(6).reshape((6, 1))
        self.P_hat = np.eye(6)

    def step(self):
        if self.q_real > 0:
            N  = 100
            dt = self.T / N
            sigma = self.q_real * np.sqrt(dt)
            for _ in range(N):
                self.v += self.rng.normal(0, sigma, (3, 1))
                self.x += self.v * dt
        else:
            self.x += self.v * T

        z = self.x + self.rng.normal(0.0, self.R, (3, 1))

        # forward-prediction
        self.x_hat = self.F @ self.x_hat
        self.P_hat = self.F @ self.P_hat @ self.F.T + self.Q

        x_fwd, P_fwd = np.copy(self.x_hat), np.copy(self.P_hat)

        # innovation covariance
        # S = H*P*H + R
        S = self.H @ self.P_hat @ self.H.T + np.eye(3) * self.R

        # Kalman gain
        # K = P*H (H*P*H + R)^-1
        K = self.P_hat @ self.H.T @ np.linalg.inv(S)

        innovation = z - self.H @ self.x_hat

        # filtered state (mean)
        # X = X + K(z - H*X)
        self.x_hat += K @ innovation

        # filtered state (covariance)
        # P = P - K*S*K
        self.P_hat -= K @ S @ K.T

        return x_fwd, P_fwd, np.copy(self.x)

In [ ]:
def one(r, n=400):
    x_trace, P_trace, truth = [], [], []
    for _ in range(n):
        x_fwd, P_fwd, x_true = r.step()
        x_trace.append(x_fwd)
        P_trace.append(P_fwd)
        truth.append(x_true)
    
    return np.array(x_trace), np.array(P_trace), np.array(truth)

In [ ]:
r = Runner()
#r.q = 0
x_trace, P_trace, truth = one(r)

In [ ]:
run_count = 1
dim       = 3
ci_mean   = sp.stats.chi2.ppf([0.025, 0.975], run_count * dim) / run_count

In [ ]:
e = evaluate_nees(np.expand_dims(x_trace[:,:3,:], 0), np.expand_dims(P_trace[:,:3,:3], 0), truth.squeeze())

In [ ]:
fig = ex.scatter(y=e.scores.mean(axis=0))
fig.add_hline(y=ci_mean[0], line_width=.5, line_dash="dash", line_color="red")
fig.add_hline(y=ci_mean[1], line_width=.5, line_dash="dash", line_color="red")
fig

In [ ]:
def many(m=200, n=400):
    x_trace, P_trace, truth_trace = [], [], []
    for _ in range(m):
        r = Runner()
        r.reset(q_model=2, q_real=2)
        x_hat, P_hat, truth = one(r, n=n)
        x_trace.append(x_hat)
        P_trace.append(P_hat)
        truth_trace.append(truth)
    
    return np.array(x_trace), np.array(P_trace), np.array(truth_trace)

In [ ]:
run_count = 100
dim       = 3
ci_mean   = sp.stats.chi2.ppf([0.025, 0.975], run_count * dim) / run_count

In [ ]:
x_trace, P_trace, truth = many(run_count, n=200)

In [ ]:
e = evaluate_nees(x_trace[:, :,:3,:], P_trace[:, :,:3,:3], truth)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=np.mean(e.scores, axis=0)[25:], mode='lines'))
#fig.add_trace(go.Scatter(y=e.scores[25:], mode='lines'))
fig.add_hline(y=ci_mean[0], line_width=.5, line_dash="dash", line_color="red")
fig.add_hline(y=ci_mean[1], line_width=.5, line_dash="dash", line_color="red")
fig

In [ ]:
ci_mean

In [ ]:
ms = e.scores.mean(axis=0)[50:]
np.mean(np.logical_and(ci_mean[0] <= ms, ms <= ci_mean[1]))